In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import Toplevel
import tkinter.messagebox
import customtkinter as ctk
from customtkinter.windows.widgets import CTkFrame, CTkLabel
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg as FigureCanvas
from  matplotlib.colors import LinearSegmentedColormap
import csv
import pandas as pd
import requests
from PIL import Image
from datetime import datetime, timedelta
from sklearn.cluster import KMeans

ctk.set_appearance_mode("light")  # Modes: "System" (standard), "Dark", "Light"
ctk.set_default_color_theme("./custom_dark_blue.json")  # Themes: "blue" (standard), "green", "dark-blue"

class CircularPlotWidget(CTkFrame):
    def __init__(self, parent=None):
        super().__init__(parent)

        # Create a colormap based on circular statistics
        colormap = LinearSegmentedColormap.from_list('rg', ["green", "yellow", "red"], N=256)

        # Create a CTkLabel for the title
        #title_label = CTkLabel(master=parent, text='Total force on the ship hull at Wharf 3A', font=("roboto", 16))
        #title_label.grid(row=0, column=2, pady=20, padx=20, sticky="nsew")

        # Create a Matplotlib figure and configure the polar plot
        self.figure, self.ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})
        self.ax.set_theta_zero_location('N')
        self.ax.set_thetamin(0)
        self.ax.set_thetamax(360)
        self.ax.set_theta_direction(-1)
        self.ax.set_rmax(1)
        self.ax.set_yticklabels([])

        # Create the colorbar using the ScalarMappable
        self.cbar = plt.colorbar(plt.cm.ScalarMappable(cmap=colormap, norm=mpl.colors.Normalize(vmin=0, vmax=5)), ax=self.ax)
        self.cbar.set_label('Risk Estimation')

        # Add legend for Port/Ship Angle
        port_angle_1 = self.ax.plot([math.radians(205), math.radians(205)], [0, 5], color='navy', linestyle='-.', label='Port/Ship Angle')
        self.ax.plot([math.radians(25), math.radians(25)], [0, 5], color='navy', linestyle='-.', label='Port/Ship Angle')

        # Set the title of the circular plot
        #self.ax.set_title('Total force on the ship hull at Wharf 3A', va='bottom', fontsize=14)

        # Create a FigureCanvasQTAgg for Matplotlib canvas
        self.canvas = FigureCanvas(self.figure, master=parent)
        self.canvas.get_tk_widget().grid(row=1, column=2, pady=20, padx=20, sticky="nsew")


    def plot_circular_data_main(self, ship_direction, total_force_direction, total_force, max_total_force, risk_color, mag):
        self.ax.clear()

        # Generate example circular data (angles in degrees)
        angles_deg = np.random.uniform(0, 360, 100)
        magnitudes = mag
        max_magnitudes = 5

        # Calculate circular statistics
        Fd_angle = np.radians(total_force_direction)
        port_angle_1 = np.radians(ship_direction)

        if 0 <= port_angle_1 < 180:
            Port_angle_2 = abs(port_angle_1 + math.pi)
        elif port_angle_1 < 360:
            Port_angle_2 = abs(port_angle_1 - math.pi)
        else:
            raise ValueError("Error: Please reenter data")

        Fd_angle_rad = Fd_angle + np.radians(np.random.uniform(0, 20, 20))
        std_dev = np.sqrt(-2 * np.log(np.sqrt(np.mean(np.sin(Fd_angle_rad)) ** 2 + np.mean(np.cos(Fd_angle_rad)) ** 2)))

        # Plot total force direction
        port_angle = self.ax.plot([port_angle_1, port_angle_1], [0, max_magnitudes], color='navy', linestyle='-.', label='Port/Ship Angle')
        self.ax.plot([Port_angle_2, Port_angle_2], [0, max_magnitudes], color='navy', linestyle='-.', label='Port/Ship Angle')
        fill_graph_area = self.ax.fill_between([Fd_angle - std_dev, Fd_angle + std_dev], 0, magnitudes, color=risk_color, alpha=1)

        # Add text annotation for F_Total_D value
        """self.ax.annotate(f'Direction: {math.degrees(Fd_angle):.2f}°\nForce: {total_force:.2f} Ton(s)\nMax Force: {max_total_force:.2f} Ton(s)',
                         xy=(Fd_angle, 1), xytext=(Fd_angle, 0.5),
                         ha='center', va='center', arrowprops=dict(arrowstyle="->"), color='black')"""

        # Configure polar plot
        self.ax.set_theta_zero_location('N')
        self.ax.set_thetamin(0)
        self.ax.set_thetamax(360)
        self.ax.set_theta_direction(-1)
        self.ax.set_rmax(1)
        self.ax.set_yticklabels([])

        # Add legend with handles and labels
        legend_handles = [port_angle[0], fill_graph_area]  # Include the fill_between artist in the legend_handles
        legend_labels = ['Port/Ship Angle', 'Total Force']  # Add a label for the fill_between
        self.ax.legend(legend_handles, legend_labels, loc='upper left')

        # Set the title of the circular plot
        self.ax.set_title('Total force on the ship hull at Wharf 3A', va='bottom', fontsize=12)

        # Redraw canvas
        self.canvas.draw()

    def plot_circular_data_backup(self, ship_direction, wind_wave_direction, wc_direction, total_force_direction, total_force, max_total_force, risk_color, mag):
        
        self.ax.clear()

        # Generate example circular data (angles in degrees)
        angles_deg = np.random.uniform(0, 360, 100)
        magnitudes = mag
        max_magnitudes = 5

        # Calculate circular statistics
        Fd_angle = np.radians(total_force_direction)
        port_angle_1 = np.radians(ship_direction)
        wind_wave_d_1 = np.radians(wind_wave_direction)
        wc_direction_1 = np.radians(wc_direction)

        if 0 <= port_angle_1 < 180:
            Port_angle_2 = abs(port_angle_1 + math.pi)
        elif port_angle_1 < 360:
            Port_angle_2 = abs(port_angle_1 - math.pi)
        else:
            raise ValueError("Error: Please enter a number")

        Fd_angle_rad = Fd_angle + np.radians(np.random.uniform(0, 20, 20))
        std_dev = np.sqrt(-2 * np.log(np.sqrt(np.mean(np.sin(Fd_angle_rad)) ** 2 + np.mean(np.cos(Fd_angle_rad)) ** 2)))

        # Plot total force direction
        port_angle = self.ax.plot([port_angle_1, port_angle_1], [0, max_magnitudes], color='navy', linestyle='-.', label='Port/Ship Angle')
        self.ax.plot([Port_angle_2, Port_angle_2], [0, max_magnitudes], color='navy', linestyle='-.', label='Port/Ship Angle')
        fill_graph_area = self.ax.fill_between([Fd_angle - std_dev, Fd_angle + std_dev], 0, magnitudes, color=risk_color, alpha=1)
        wind_angle_1 = self.ax.plot([wind_wave_d_1, wind_wave_d_1], [0, max_magnitudes], color='yellowgreen', linestyle='-.', label='Wind-Wave Angle')
        wc_angle_1 = self.ax.plot([wc_direction_1, wc_direction_1], [0, max_magnitudes], color='yellow', linestyle='-.', label='Sea Current Angle')

        # Add text annotation for F_Total_D value
        """self.ax.annotate(f'Direction: {math.degrees(Fd_angle):.2f}°\nForce: {total_force:.2f} Ton(s)\nMax Force: {max_total_force:.2f} Ton(s)',
                         xy=(Fd_angle, 1), xytext=(Fd_angle, 0.5),
                         ha='center', va='center', arrowprops=dict(arrowstyle="->"), color='black')"""

        # Configure polar plot
        self.ax.set_theta_zero_location('N')
        self.ax.set_thetamin(0)
        self.ax.set_thetamax(360)
        self.ax.set_theta_direction(-1)
        self.ax.set_rmax(1)
        self.ax.set_yticklabels([])

        # Add legend with handles and labels
        legend_handles = [port_angle[0], fill_graph_area, wind_angle_1[0], wc_angle_1[0]]  # Include the fill_between artist in the legend_handles
        legend_labels = ['Port/Ship Angle', 'Total Force', 'Wind-Wave Angle', 'Sea Current Angle']  # Add a label for the fill_between
        self.ax.legend(legend_handles, legend_labels, loc='upper left')

        # Set the title of the circular plot
        self.ax.set_title('Total force on the ship hull at Wharf 3A', va='bottom', fontsize=12)

        # Redraw canvas
        self.canvas.draw()


class App(ctk.CTk):
    def __init__(self):
        super().__init__()

        # configure window
        self.title("Berthing&Unberthing Risk Estimation Application")
        self.geometry(f"{1280}x{720}")

        # configure grid layout (4x4)
        self.grid_columnconfigure(1, weight=1)
        #self.grid_columnconfigure((2, 3), weight=0)
        self.grid_rowconfigure(0, weight=1)

        # create sidebar frame with widgets
        self.sidebar_frame = ctk.CTkFrame(self, width=140, corner_radius=0, fg_color="grey75", border_color="#a0a0a0")
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(5, weight=1)
        self.logo_label = ctk.CTkLabel(self.sidebar_frame, text="Risk Estimation\nApplication W.3A", font=ctk.CTkFont(size=20, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        #self.sidebar_button_1 = ctk.CTkButton(self.sidebar_frame, text="Import Ship Data CSV", command=self.import_and_filter_data)
        #self.sidebar_button_1.grid(row=1, column=0, padx=20, pady=10)
        self.sidebar_button_2 = ctk.CTkButton(self.sidebar_frame, text="Current Force Calculate", command=self.current_force_calculate)
        self.sidebar_button_2.grid(row=1, column=0, padx=20, pady=10)
        self.sidebar_button_3 = ctk.CTkButton(self.sidebar_frame, text="1-3 hr. API Forecast Force Calculate", command=self.first_forecast_force_calculate)
        self.sidebar_button_3.grid(row=2, column=0, padx=20, pady=10)
        self.sidebar_button_4 = ctk.CTkButton(self.sidebar_frame, text="Export Result", command=self.export_graph_data)
        self.sidebar_button_4.grid(row=3, column=0, padx=20, pady=10)
        self.appearance_mode_label = ctk.CTkLabel(self.sidebar_frame, text="Appearance Mode:", anchor="w")
        self.appearance_mode_label.grid(row=7, column=0, padx=20, pady=(10, 0))
        self.appearance_mode_optionemenu = ctk.CTkOptionMenu(self.sidebar_frame, values=["Light", "Dark", "System"],
                                                                       command=self.change_appearance_mode_event)
        self.appearance_mode_optionemenu.grid(row=8, column=0, padx=20, pady=(10, 10))

        """self.optionmenu_1 = ctk.CTkOptionMenu(self.sidebar_frame, dynamic_resizing=False,
                                                        values=["Value 1", "Value 2", "Value Long Long Long"])
        self.optionmenu_1.grid(row=5, column=0, padx=20, pady=(20, 10))"""

        self.string_input_button = ctk.CTkButton(self.sidebar_frame, text="Import OpenWeather API Key",
                                                           command=self.open_APIKey_input_dialog)
        self.string_input_button.grid(row=6, column=0, padx=20, pady=(10, 10))

        """self.main_frame = ctk.CTkFrame(self, width=1226, corner_radius=0)
        self.main_frame.grid(row=0, column=1, rowspan=4, sticky="nsew")
        self.main_frame.grid_columnconfigure(1, weight=1)
        self.main_frame.grid_rowconfigure(4, weight=1)"""

        # Create a function to update the label with the current date and time
        def update_time_label():
            self.current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            date_label = ctk.CTkLabel(self.sidebar_frame)
            date_label.grid(row=9, column=0, padx=20, pady=20)
            date_label.configure(text=self.current_time)
            date_label.after(1000, update_time_label)  # Update every 1000 ms (1 second)

        update_time_label()

        # create tabview
        self.tabview = ctk.CTkTabview(self, width=200)
        self.tabview.grid(row=0, column=1, padx=(0, 0), pady=(0, 0), sticky="nsew")
        self.tabview.add("Data Input/Output")
        self.tabview.add("Main Graph")
        self.tabview.add("Back Up Graph")
        self.tabview.add("Real Time History Data")
        self.tabview.add("API History Data")
        
        #self.tabview.tab("Data Input/Output").grid_columnconfigure((1,3),  weight=0)  # configure grid of individual tabs
        #self.tabview.tab("Data Input/Output").grid_rowconfigure((1,16), weight=0)

        # Create a CTkFrame as a container with a border color
        self.container_frame_1 = ctk.CTkFrame(self.tabview.tab("Data Input/Output"), fg_color="grey75", border_color="#a0a0a0")
        self.container_frame_1.grid(row=0, column=0, padx=20, pady=5, sticky="nsew")
        self.container_frame_1.grid_columnconfigure((1,3),  weight=0)
        self.container_frame_1.grid_rowconfigure((1,5), weight=0)

        self.container_frame_2 = ctk.CTkFrame(self.tabview.tab("Data Input/Output"), fg_color="grey75", border_color="#a0a0a0")
        self.container_frame_2.grid(row=1, column=0, padx=20, pady=5, sticky="nsew")
        self.container_frame_2.grid_columnconfigure((1,3),  weight=0)
        self.container_frame_2.grid_rowconfigure((1,16), weight=0)

        self.container_frame_3 = ctk.CTkFrame(self.tabview.tab("Data Input/Output"), fg_color="grey75", border_color="#a0a0a0")
        self.container_frame_3.grid(row=2, column=0, padx=20, pady=5, sticky="nsew")
        self.container_frame_3.grid_columnconfigure((1,3),  weight=0)
        self.container_frame_3.grid_rowconfigure((1,16), weight=0)

        self.container_frame_4 = ctk.CTkFrame(self.tabview.tab("Data Input/Output"), fg_color="grey75", border_color="#a0a0a0")
        self.container_frame_4.grid(row=3, column=0, padx=20, pady=5, sticky="nsew")
        self.container_frame_4.grid_columnconfigure((1,3),  weight=0)
        self.container_frame_4.grid_rowconfigure((1,16), weight=0)

        self.container_frame_5 = ctk.CTkFrame(self.tabview.tab("Data Input/Output"), fg_color="grey75", border_color="#a0a0a0")
        self.container_frame_5.grid(row=4, column=0, padx=20, pady=5, sticky="nsew")
        self.container_frame_5.grid_columnconfigure((1,3),  weight=0)
        self.container_frame_5.grid_rowconfigure((1,16), weight=0)

        self.label_1 = ctk.CTkLabel(self.container_frame_1, text="Ship Data")
        self.label_1.grid(row=0, column=0, padx=20, pady=5)

        self.label_6 = ctk.CTkLabel(self.container_frame_1, text="Ship Name")
        self.label_6.grid(row=0, column=1, padx=20, pady=5)
        self.label_7 = ctk.CTkLabel(self.container_frame_1, text="Direction (Deg)")
        self.label_7.grid(row=0, column=2, padx=20, pady=5)
        self.label_8 = ctk.CTkLabel(self.container_frame_1, text="LOA (m)")
        self.label_8.grid(row=0, column=3, padx=20, pady=5)
        self.label_9 = ctk.CTkLabel(self.container_frame_1, text="Beam (m)")
        self.label_9.grid(row=2, column=2, padx=20, pady=5)
        self.label_10 = ctk.CTkLabel(self.container_frame_1, text="LBP (m)")
        self.label_10.grid(row=2, column=1, padx=20, pady=5)
        self.label_11 = ctk.CTkLabel(self.container_frame_1, text="Draft (m)")
        self.label_11.grid(row=4, column=2, padx=20, pady=5)
        self.label_12 = ctk.CTkLabel(self.container_frame_1, text="Freeboard (m)")
        self.label_12.grid(row=4, column=3, padx=20, pady=5)
        self.label_13 = ctk.CTkLabel(self.container_frame_1, text="Displacement\n(Deadweight + Light Ship) (Ton)")
        self.label_13.grid(row=4, column=1, padx=20, pady=5)
        self.label_25 = ctk.CTkLabel(self.container_frame_1, text="Depth/Height (m)")
        self.label_25.grid(row=2, column=3, padx=20, pady=5)

        self.label_2 = ctk.CTkLabel(self.container_frame_2, text="Wind Data")
        self.label_2.grid(row=0, column=0, padx=20, pady=5)

        self.label_14 = ctk.CTkLabel(self.container_frame_2, text="Speed (Knot)")
        self.label_14.grid(row=0, column=1, padx=20, pady=5)
        self.label_15 = ctk.CTkLabel(self.container_frame_2, text="Direction (Deg)")
        self.label_15.grid(row=0, column=2, padx=20, pady=5)

        self.label_3 = ctk.CTkLabel(self.container_frame_3, text="Wave Data")
        self.label_3.grid(row=0, column=0, padx=20, pady=5)

        self.label_16 = ctk.CTkLabel(self.container_frame_3, text="Speed (Knot)")
        self.label_16.grid(row=0, column=1, padx=20, pady=5)
        self.label_17 = ctk.CTkLabel(self.container_frame_3, text="Wave Height (m)")
        self.label_17.grid(row=0, column=2, padx=20, pady=5)
        self.label_18 = ctk.CTkLabel(self.container_frame_3, text="Direction (Deg)")
        self.label_18.grid(row=0, column=3, padx=20, pady=5)

        self.label_4 = ctk.CTkLabel(self.container_frame_4, text="Sea Current Data")
        self.label_4.grid(row=0, column=0, padx=20, pady=5)

        self.label_19 = ctk.CTkLabel(self.container_frame_4, text="Height Change Rate \nCompare to Previous Hour (m)")
        self.label_19.grid(row=0, column=1, padx=20, pady=5)
        self.label_20 = ctk.CTkLabel(self.container_frame_4, text="Speed (Knot)")
        self.label_20.grid(row=0, column=2, padx=20, pady=5)
        self.label_21 = ctk.CTkLabel(self.container_frame_4, text="Direction (Deg)")
        self.label_21.grid(row=0, column=3, padx=20, pady=5)

        self.label_5 = ctk.CTkLabel(self.container_frame_5, text="Result")
        self.label_5.grid(row=0, column=0, padx=20, pady=5)

        self.label_22 = ctk.CTkLabel(self.container_frame_5, text="Total Force (Ton)")
        self.label_22.grid(row=0, column=1, padx=20, pady=5)
        self.label_23 = ctk.CTkLabel(self.container_frame_5, text="Direction (Deg)")
        self.label_23.grid(row=0, column=2, padx=20, pady=5)
        self.label_24 = ctk.CTkLabel(self.container_frame_5, text="Risk Estimation")
        self.label_24.grid(row=0, column=3, padx=20, pady=5)
    
        

        self.retrieve_data_button_1 = ctk.CTkButton(self.container_frame_1, text="Retrieve Ship Data", command=self.retrieve_ship_data)
        self.retrieve_data_button_1.grid(row=1, column=0, padx=20, pady=5)

        self.entry_1 = ctk.CTkEntry(self.container_frame_1, placeholder_text="Ship name", fg_color="#7EA8E8")
        self.entry_1.grid(row=1, column=1, padx=20, pady=5)
        self.entry_2 = ctk.CTkEntry(self.container_frame_1, placeholder_text="Direction (Deg)", fg_color="#7EA8E8")
        self.entry_2.grid(row=1, column=2, padx=20, pady=5)
        self.entry_3 = ctk.CTkEntry(self.container_frame_1, placeholder_text="LOA (m)")
        self.entry_3.grid(row=1, column=3, padx=20, pady=5)
        self.entry_3.configure(state="readonly")
        self.entry_4 = ctk.CTkEntry(self.container_frame_1, placeholder_text="Beam (m)")
        self.entry_4.grid(row=3, column=2, padx=20, pady=5)
        self.entry_4.configure(state="readonly")
        self.entry_5 = ctk.CTkEntry(self.container_frame_1, placeholder_text="LBP (m)", fg_color="#7EA8E8")
        self.entry_5.grid(row=3, column=1, padx=20, pady=5)
        self.entry_6 = ctk.CTkEntry(self.container_frame_1, placeholder_text="Draft (m)", fg_color="#7EA8E8")
        self.entry_6.grid(row=5, column=2, padx=20, pady=5)
        self.entry_7 = ctk.CTkEntry(self.container_frame_1, placeholder_text="Freeboard (m)")
        self.entry_7.grid(row=5, column=3, padx=20, pady=5)
        self.entry_7.configure(state="readonly")
        self.entry_8 = ctk.CTkEntry(self.container_frame_1, placeholder_text="Displacement (Ton)", fg_color="#7EA8E8")
        self.entry_8.grid(row=5, column=1, padx=20, pady=5)

        self.entry_9 = ctk.CTkEntry(self.container_frame_2, placeholder_text="Speed (Knot)", fg_color="#7EA8E8")
        self.entry_9.grid(row=1, column=1, padx=20, pady=5)
        self.entry_10 = ctk.CTkEntry(self.container_frame_2, placeholder_text="Direction (Deg)", fg_color="#7EA8E8")
        self.entry_10.grid(row=1, column=2, padx=20, pady=5)

        self.entry_11 = ctk.CTkEntry(self.container_frame_3, placeholder_text="Speed (Knot)")
        self.entry_11.grid(row=1, column=1, padx=20, pady=5)
        self.entry_11.configure(state="readonly")
        self.entry_12 = ctk.CTkEntry(self.container_frame_3, placeholder_text="Wave Height (m)")
        self.entry_12.grid(row=1, column=2, padx=20, pady=5)
        self.entry_12.configure(state="readonly")
        self.entry_13 = ctk.CTkEntry(self.container_frame_3, placeholder_text="Direction (Deg)")
        self.entry_13.grid(row=1, column=3, padx=20, pady=5)
        self.entry_13.configure(state="readonly")

        self.entry_14 = ctk.CTkEntry(self.container_frame_4, width=165, placeholder_text="Height Change Rate (m)")
        self.entry_14.grid(row=1, column=1, padx=20, pady=5)
        self.entry_14.configure(state="readonly")
        self.entry_15 = ctk.CTkEntry(self.container_frame_4, placeholder_text="Speed (knot)")
        self.entry_15.grid(row=1, column=2, padx=20, pady=5)
        self.entry_15.configure(state="readonly")
        self.entry_16 = ctk.CTkEntry(self.container_frame_4, placeholder_text="Direction (Deg)")
        self.entry_16.grid(row=1, column=3, padx=20, pady=5)
        self.entry_16.configure(state="readonly")

        self.entry_17 = ctk.CTkEntry(self.container_frame_5, placeholder_text="Total Force (Ton)", fg_color="#80BF80")
        self.entry_17.grid(row=1, column=1, padx=20, pady=5)
        self.entry_17.configure(state="readonly")
        self.entry_18 = ctk.CTkEntry(self.container_frame_5, placeholder_text="Direction (Deg)", fg_color="#80BF80")
        self.entry_18.grid(row=1, column=2, padx=20, pady=5)
        self.entry_18.configure(state="readonly")
        self.entry_19 = ctk.CTkEntry(self.container_frame_5, width=200, placeholder_text="Risk Estimation", fg_color="#80BF80")
        self.entry_19.grid(row=1, column=3, padx=20, pady=5)
        self.entry_19.configure(state="readonly")

        self.entry_20 = ctk.CTkEntry(self.container_frame_1, placeholder_text="Ship Depth/Height (m)")
        self.entry_20.grid(row=3, column=3, padx=20, pady=5)
        self.entry_20.configure(state="readonly")

        self.entry_21 = ctk.CTkEntry(self.container_frame_2, placeholder_text="API Date&Time")
        self.entry_21.grid(row=1, column=3, padx=20, pady=5)
        self.entry_21.configure(state="readonly")
        
        self.canvas_bar = tk.Canvas(self.container_frame_5, width=300, height=100)
        self.canvas_bar.grid(row=2, column=3, padx=20, pady=5)

        
        # Inside Tab 2
        self.background_image_1 = tk.PhotoImage(file=".\Lay out Port_TLMM_W.3A.png")  # Replace with your image file path
        self.background_label_2 = ctk.CTkLabel(self.tabview.tab("Main Graph"), text="", image=self.background_image_1)
        self.background_label_2.grid(row=1, column=3, pady=20, padx=20, sticky="nsew")  # Fill the entire window
        
        self.tabview.tab("Main Graph").grid_columnconfigure((1,3) , weight=1)
        self.tabview.tab("Main Graph").grid_rowconfigure(1 , weight=1)
        self.title_label_1 = CTkLabel(self.tabview.tab("Main Graph"), text='Total force on the ship hull at Wharf 3A', font=("roboto", 16))
        self.title_label_1.grid(row=0, column=2, pady=20, padx=20, sticky="nsew")
        self.circular_plot_widget_tab2 = CircularPlotWidget(self.tabview.tab("Main Graph"))
        #circular_plot_widget_tab2.grid(row=0, column=2, pady=20, padx=20, sticky="nsew")
        self.result_label_1 = ctk.CTkLabel(self.tabview.tab("Main Graph"), text="Total Force : 0 Ton", font=ctk.CTkFont(size=15, weight="bold"))
        self.result_label_1.grid(row=2, column=3, padx=20, pady=5)
        self.result_label_2 = ctk.CTkLabel(self.tabview.tab("Main Graph"), text="Direction : - Deg.", font=ctk.CTkFont(size=15, weight="bold"))
        self.result_label_2.grid(row=3, column=3, padx=20, pady=5)
        self.result_label_3 = ctk.CTkLabel(self.tabview.tab("Main Graph"), text="Risk Estimation : -", font=ctk.CTkFont(size=15, weight="bold"))
        self.result_label_3.grid(row=4, column=3, padx=20, pady=5)

        # Inside Tab 3
        self.background_label_3 = ctk.CTkLabel(self.tabview.tab("Back Up Graph"), text="", image=self.background_image_1)
        self.background_label_3.grid(row=1, column=3, pady=20, padx=20, sticky="nsew")  # Fill the entire window
        self.tabview.tab("Back Up Graph").grid_columnconfigure((1,3), weight=1)
        self.tabview.tab("Back Up Graph").grid_rowconfigure(1 , weight=1)
        self.title_label_2 = CTkLabel(self.tabview.tab("Back Up Graph"), text='Total force on the ship hull at Wharf 3A (Back Up)', font=("roboto", 16))
        self.title_label_2.grid(row=0, column=2, pady=20, padx=20, sticky="nsew")
        self.circular_plot_widget_tab3 = CircularPlotWidget(self.tabview.tab("Back Up Graph"))

        self.result_label_4 = ctk.CTkLabel(self.tabview.tab("Back Up Graph"), text="Total Force : 0 Ton", font=ctk.CTkFont(size=15, weight="bold"))
        self.result_label_4.grid(row=2, column=3, padx=20, pady=5)
        self.result_label_5 = ctk.CTkLabel(self.tabview.tab("Back Up Graph"), text="Direction : - Deg.", font=ctk.CTkFont(size=15, weight="bold"))
        self.result_label_5.grid(row=3, column=3, padx=20, pady=5)
        self.result_label_6 = ctk.CTkLabel(self.tabview.tab("Back Up Graph"), text="Risk Estimation : -", font=ctk.CTkFont(size=15, weight="bold"))
        self.result_label_6.grid(row=4, column=3, padx=20, pady=5)

        # Inside Tab 4
        # Load data from CSV
        self.csv_path_1 = './Output_Real_Time_Database.csv'
        self.df_1 = pd.read_csv(self.csv_path_1)

        # Create a frame for the table
        table_frame = ttk.Frame(self.tabview.tab("Real Time History Data"))

        # Create a Treeview widget for the table
        self.treeview = ttk.Treeview(table_frame, columns=list(self.df_1.columns), show="headings")

        # Add headers to the Treeview
        for col in self.df_1.columns:
            self.treeview.heading(col, text=col)

        # Insert data into the Treeview
        for index, row in self.df_1.iterrows():
            self.treeview.insert("", "end", values=list(row))

        # Create a horizontal scrollbar
        x_scrollbar = ttk.Scrollbar(table_frame, orient=tk.HORIZONTAL, command=self.treeview.xview)

        # Set the scrollbar to the Treeview
        self.treeview.configure(xscrollcommand=x_scrollbar.set)

        # Pack the Treeview and scrollbar
        self.treeview.pack(fill=tk.BOTH, expand=True)
        x_scrollbar.pack(fill=tk.X)

        # Pack the frame
        table_frame.pack(fill=tk.BOTH, expand=True)
        
        # Schedule auto-update every 1000 milliseconds (1 second)
        self.after(1000, self.auto_update_table_1)
        
        # Inside Tab 5
        # Load data from CSV
        self.csv_path_2 = './Output_API_Database.csv'
        self.df_2 = pd.read_csv(self.csv_path_2)

        # Create a frame for the table
        table_frame = ttk.Frame(self.tabview.tab("API History Data"))

        # Create a Treeview widget for the table
        self.treeview = ttk.Treeview(table_frame, columns=list(self.df_2.columns), show="headings")

        # Add headers to the Treeview
        for col in self.df_2.columns:
            self.treeview.heading(col, text=col)

        # Insert data into the Treeview
        for index, row in self.df_2.iterrows():
            self.treeview.insert("", "end", values=list(row))

        # Create a horizontal scrollbar
        x_scrollbar = ttk.Scrollbar(table_frame, orient=tk.HORIZONTAL, command=self.treeview.xview)

        # Set the scrollbar to the Treeview
        self.treeview.configure(xscrollcommand=x_scrollbar.set)

        # Pack the Treeview and scrollbar
        self.treeview.pack(fill=tk.BOTH, expand=True)
        x_scrollbar.pack(fill=tk.X)

        # Pack the frame
        table_frame.pack(fill=tk.BOTH, expand=True)

        # Schedule auto-update every 1000 milliseconds (1 second)
        self.after(1000, self.auto_update_table_2)


        #set appearance_mode_optionemenu
        self.appearance_mode_optionemenu.set("Light")
        #self.optionmenu_1.set("Option")

        # Initialize a variable to keep track of the bar item
        self.bar_item = None
        
        self.canvas_bar.create_rectangle(30, 40, 270, 70, fill="red")
        self.canvas_bar.create_rectangle(30, 40, 230, 70, fill="darkorange")
        self.canvas_bar.create_rectangle(30, 40, 190, 70, fill="yellow")
        self.canvas_bar.create_rectangle(30, 40, 150, 70, fill="greenyellow")
        self.canvas_bar.create_rectangle(30, 40, 110, 70, fill="limegreen")
        self.canvas_bar.create_rectangle(30, 40, 70, 70, fill="green")

    def auto_update_table(self, csv_path):
        max_rows = 1000

        # Read all rows from the CSV file
        with open(csv_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            all_rows = list(csv_reader)

        # Clear existing rows in the Treeview
        for item in self.treeview.get_children():
            self.treeview.delete(item)

        # Insert new data into the Treeview
        for index, row in enumerate(all_rows):
            self.treeview.insert("", "end", values=row)

        # Check if the number of rows exceeds the maximum
        if len(all_rows) > max_rows:
            # Open the CSV file in write mode
            with open(csv_path, 'w', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)

                # Write all rows except the oldest ones
                csv_writer.writerows(all_rows[-max_rows:])

        # Schedule the next auto-update
        self.after(1000, lambda: self.auto_update_table(csv_path))

    def auto_update_table_1(self):
        self.auto_update_table(self.csv_path_1)

    def auto_update_table_2(self):
        self.auto_update_table(self.csv_path_2)

    def update_colored_bar(self, x, y, width, height, color):
        # Delete the old bar if it exists
        if self.bar_item:
            self.canvas_bar.delete(self.bar_item)

        # Draw a new colored bar
        self.bar_item = self.canvas_bar.create_rectangle(x, y, x + width, y + height, fill=color)

    def open_APIKey_input_dialog(self):
        dialog = ctk.CTkInputDialog(text="Input OpenWeather API Key:", title="Import OpenWeather API Key")
        self.api_key = dialog.get_input()
        #print("CTkInputDialog:", dialog.get_input())

    def retrieve_ship_data(self):
        self.file_path_1 = pd.read_csv("./Master Vessel for TLMM Model Plant Innovation.csv")
        #file_path_2 = self.file_path_1
        name_to_search = str.upper(self.entry_1.get())

        if not self.file_path_1.empty and name_to_search:
            try:
                found = False
                
                for index, row in self.file_path_1.iterrows():
                    if row["VehicleName"] == name_to_search:
                        self.entry_1.delete(0, "end")
                        self.entry_1.insert(0, row['VehicleName'])
                        self.entry_3.configure(state="normal") # Set the entry to normal state to allow modification
                        self.entry_3.delete(0, "end")
                        self.entry_3.insert(0, row['BaseLength'])
                        self.entry_3.configure(state="readonly") # Set the entry back to readonly state
                        self.entry_4.configure(state="normal") # Set the entry to normal state to allow modification
                        self.entry_4.delete(0, "end")
                        self.entry_4.insert(0, row['BaseWidth'])
                        self.entry_4.configure(state="readonly") # Set the entry back to readonly state
                        self.entry_20.configure(state="normal") # Set the entry to normal state to allow modification
                        self.entry_20.delete(0, "end")
                        self.entry_20.insert(0, row['BaseHeight'])
                        self.entry_20.configure(state="readonly") # Set the entry back to readonly state

                        found = True
                        break

                if not found:
                    tkinter.messagebox.showwarning("Warning", "Name not found in the CSV file.")
                    
            except Exception as e:
                error_message = f"Failed to find ship data.\nError: {str(e)}"
                tkinter.messagebox.showerror("Error", error_message)
        else:
            tkinter.messagebox.showwarning("Warning", "Please enter a file path and a name.")
                
    def change_appearance_mode_event(self, new_appearance_mode: str):
        ctk.set_appearance_mode(new_appearance_mode)

    """def change_scaling_event(self, new_scaling: str):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        ctk.set_widget_scaling(new_scaling_float)"""
    

    def wind_calculate(self,Wind_D,Air_Pressure,Wind_Speed_MS,Freeboard_Area_Stern,Freeboard_Area_Side,Freeboard_Area_PB_SB,Bow_D,Ship_D):
        if 0 < Wind_D < 180 :
            Wind_Dt = abs(Wind_D + 180)
        elif Wind_D < 360 :
            Wind_Dt = abs(Wind_D - 180)
        else :
            raise ValueError("Error: Failed to calculate wind force.\nPlease reenter wind data")
        
        Wind_Theta = abs(Wind_Dt-Ship_D)
            
        if Wind_Theta <= 180:
            Wind_Theta = Wind_Theta
        elif Wind_Theta < 360:
            Wind_Theta = abs(Wind_Theta-360)

        if Wind_Theta == 0:
            F_Wind = 0.5 * Air_Pressure * pow(Wind_Speed_MS,2) * Freeboard_Area_Stern
            F_Wind_X = F_Wind * math.sin(math.radians(Wind_Dt))
            F_Wind_Y = F_Wind * math.cos(math.radians(Wind_Dt))

        elif 0 < Wind_Theta < 90:
            F_Wind = 0.5 * Air_Pressure * pow(Wind_Speed_MS,2) * ((math.sin(math.radians(Wind_Theta)) * Freeboard_Area_Side) + 
                                                                    (math.cos(math.radians(Wind_Theta)) * Freeboard_Area_Stern))
            F_Wind_X = F_Wind * math.sin(math.radians(Wind_Dt))
            F_Wind_Y = F_Wind * math.cos(math.radians(Wind_Dt))

        elif Wind_Theta == 90:
            F_Wind = 0.5 * Air_Pressure * pow(Wind_Speed_MS,2) * (Freeboard_Area_Side + (math.sin(math.radians(Wind_Theta + (Bow_D/2)))) * Freeboard_Area_PB_SB)
            F_Wind_X = F_Wind * math.sin(math.radians(Wind_Dt))
            F_Wind_Y = F_Wind * math.cos(math.radians(Wind_Dt))

        elif 90 < Wind_Theta < 180:
            if 0 < (180-Wind_Theta) + (Bow_D/2) < 90:
                F_Wind = 0.5 * Air_Pressure * pow(Wind_Speed_MS,2) * ((math.sin(math.radians(Wind_Theta)) * Freeboard_Area_Side) + 
                                                                        (math.sin(math.radians((180-Wind_Theta) + (Bow_D/2))) * Freeboard_Area_PB_SB))
            elif (180-Wind_Theta) + (Bow_D/2) == 90:
                F_Wind = 0.5 * Air_Pressure * pow(Wind_Speed_MS,2) * ((math.sin(math.radians(Wind_Theta)) * Freeboard_Area_Side) + Freeboard_Area_PB_SB)
            elif 90 < (180-Wind_Theta) + (Bow_D/2) < 180:
                F_Wind = 0.5 * Air_Pressure * pow(Wind_Speed_MS,2) * ((math.sin(math.radians(Wind_Theta)) * Freeboard_Area_Side) + 
                                                                        (math.sin(math.radians((180-Wind_Theta) + (Bow_D/2))) * Freeboard_Area_PB_SB))
            else:
                raise ValueError("Error: Failed to calculate wind force.\nPlease reenter wind data")
            
            F_Wind_X = F_Wind * math.sin(math.radians(Wind_Dt))
            F_Wind_Y = F_Wind * math.cos(math.radians(Wind_Dt))

        elif Wind_Theta == 180:
            F_Wind = 0.5 * Air_Pressure * pow(Wind_Speed_MS,2) * math.cos(math.radians(Bow_D/2))*Freeboard_Area_PB_SB * 2
            F_Wind_X = F_Wind * math.sin(math.radians(Wind_Dt))
            F_Wind_Y = F_Wind * math.cos(math.radians(Wind_Dt))

        else :
            raise ValueError("Error: Failed to calculate wind force.\nPlease reenter wind data")

        return F_Wind_X, F_Wind_Y

    def wave_calculate(self,Wave_D,Sea_Density,Cd,Wave_Height,Wave_Speed_MS,Bow_D,Ship_D,Ship_LOA,Lf,Ship_Beam,Half_Ship_Beam):
        Wave_Draft_Area_Side = Wave_Height*(Ship_LOA-Lf) #Side
        Wave_Draft_Area_Stern = Wave_Height*Ship_Beam #Stern (Back) Area
        Wave_Draft_Area_PB_SB = math.sqrt((Lf*Lf)+(Half_Ship_Beam*Half_Ship_Beam))*Wave_Height #Ship Bow Area
        
        if 0 < Wave_D < 180 :
            Wave_Dt = abs(Wave_D + 180)
        elif Wave_D < 360 :
            Wave_Dt = abs(Wave_D - 180)
        else :
            raise ValueError("Error: Failed to calculate wave force.\nPlease reenter wind data")

        Wave_Theta = abs(Wave_Dt-Ship_D)
        
        if Wave_Theta <= 180:
            Wave_Theta = Wave_Theta
        elif Wave_Theta < 360:
            Wave_Theta = abs(Wave_Theta-360)


        if Wave_Theta == 0:
            F_Wave = 0.5 * Sea_Density * Cd * pow(Wave_Speed_MS,2) * Wave_Draft_Area_Stern
            F_Wave_X = F_Wave * math.sin(math.radians(Wave_Dt))
            F_Wave_Y = F_Wave * math.cos(math.radians(Wave_Dt))
                
        elif 0 < Wave_Theta < 90:
            F_Wave = 0.5 * Sea_Density * Cd * pow(Wave_Speed_MS,2) * ((math.sin(math.radians(Wave_Theta)) * Wave_Draft_Area_Side) + 
                                                                (math.cos(math.radians(Wave_Theta)) * Wave_Draft_Area_Stern))
            F_Wave_X = F_Wave * math.sin(math.radians(Wave_Dt))
            F_Wave_Y = F_Wave * math.cos(math.radians(Wave_Dt))

        elif Wave_Theta == 90:
            F_Wave = 0.5 * Sea_Density * Cd * pow(Wave_Speed_MS,2) * (Wave_Draft_Area_Side + (math.sin(math.radians(Wave_Theta + (Bow_D/2)))) * Wave_Draft_Area_PB_SB)
            F_Wave_X = F_Wave * math.sin(math.radians(Wave_Dt))
            F_Wave_Y = F_Wave * math.cos(math.radians(Wave_Dt))

        elif 90 < Wave_Theta < 180:
            if 0 < (180-Wave_Theta) + (Bow_D/2) < 90:
                F_Wave = 0.5 * Sea_Density * Cd * pow(Wave_Speed_MS,2) * ((math.sin(math.radians(Wave_Theta)) * Wave_Draft_Area_Side) + 
                                                                    (math.sin(math.radians((180-Wave_Theta) + (Bow_D/2))) * Wave_Draft_Area_PB_SB))
            elif (180-Wave_Theta) + (Bow_D/2) == 90:
                F_Wave = 0.5 * Sea_Density * Cd * pow(Wave_Speed_MS,2) * ((math.sin(math.radians(Wave_Theta)) * Wave_Draft_Area_Side) + Wave_Draft_Area_PB_SB)
            elif 90 < (180-Wave_Theta) + (Bow_D/2) < 180:
                    F_Wave = 0.5 * Sea_Density * Cd * pow(Wave_Speed_MS,2) * ((math.sin(math.radians(Wave_Theta)) * Wave_Draft_Area_Side) + 
                                                                    (math.sin(math.radians((180-Wave_Theta) + (Bow_D/2))) * Wave_Draft_Area_PB_SB))
            else:
                raise ValueError("Error: Failed to calculate wave force.\nPlease reenter wind data")
            
            F_Wave_X = F_Wave * math.sin(math.radians(Wave_Dt))
            F_Wave_Y = F_Wave * math.cos(math.radians(Wave_Dt))

        elif Wave_Theta == 180:
            F_Wave = 0.5 * Sea_Density * Cd * pow(Wave_Speed_MS,2) * math.cos(math.radians(Bow_D/2))*Wave_Draft_Area_PB_SB * 2
            F_Wave_X = F_Wave * math.sin(math.radians(Wave_Dt))
            F_Wave_Y = F_Wave * math.cos(math.radians(Wave_Dt))

        else :
            raise ValueError("Error: Failed to calculate wave force.\nPlease reenter wind data")

        return F_Wave_X, F_Wave_Y


    def wc_calculate(self,WC_D,Wave_Height,Sea_Density,Cd,WC_Speed_MS,Bow_D,Ship_D,Draft,LBP,Lf,Ship_Beam,Half_Ship_Beam):
        Real_Draft = Draft - Wave_Height
        Draft_Area_Side = Real_Draft*LBP #Side
        Draft_Area_Stern = Real_Draft*Ship_Beam #Stern (Back) Area
        Draft_Area_PB_SB = math.sqrt(pow(Lf,2)+pow(Half_Ship_Beam,2))*Real_Draft #Ship Bow Area

        if 0 < WC_D < 180 :
            WC_Dt = abs(WC_D + 180)
        elif WC_D < 360 :
            WC_Dt = abs(WC_D - 180)
        else :
                raise ValueError("Error: Failed to calculate sea current force.\nPlease reenter wind data")
            
        WC_Theta = abs(WC_Dt-Ship_D)
        if WC_Theta <= 180:
            WC_Theta = WC_Theta
        elif WC_Theta < 360:
            WC_Theta = abs(WC_Theta-360)

        if WC_Theta == 0:
            F_WC = 0.5 * Sea_Density * Cd * pow(WC_Speed_MS,2) * Draft_Area_Stern
            F_WC_X = F_WC * math.sin(math.radians(WC_Dt))
            F_WC_Y = F_WC * math.cos(math.radians(WC_Dt))

        elif 0 < WC_Theta < 90:
            F_WC = 0.5 * Sea_Density * Cd * pow(WC_Speed_MS,2) * (math.sin(math.radians(WC_Theta)) * Draft_Area_Side) + (math.cos(math.radians(WC_Theta)) * Draft_Area_Stern)
            F_WC_X = F_WC * math.sin(math.radians(WC_Dt))
            F_WC_Y = F_WC * math.cos(math.radians(WC_Dt))

        elif WC_Theta == 90:
            F_WC = 0.5 * Sea_Density * Cd * pow(WC_Speed_MS,2) * (Draft_Area_Side + (math.sin(math.radians(WC_Theta + (Bow_D/2)))) * Draft_Area_PB_SB)
            F_WC_X = F_WC * math.sin(math.radians(WC_Dt))
            F_WC_Y = F_WC * math.cos(math.radians(WC_Dt))

        elif 90 < WC_Theta < 180:
            if 90 < ((180-WC_Theta) + (Bow_D/2)) < 180:
                F_WC = 0.5 * Sea_Density * Cd * pow(WC_Speed_MS,2) * ((math.sin(math.radians(WC_Theta)) * Draft_Area_Side) + 
                                                                    (math.sin(math.radians((180-WC_Theta) + (Bow_D/2))) * Draft_Area_PB_SB))
            elif ((180-WC_Theta) + (Bow_D/2)) == 90:
                F_WC = 0.5 * Sea_Density * Cd * pow(WC_Speed_MS,2) * ((math.sin(math.radians(WC_Theta)) * Draft_Area_Side) + Draft_Area_PB_SB)
            elif 0 < ((180-WC_Theta) + (Bow_D/2)) < 90:
                F_WC = 0.5 * Sea_Density * Cd * pow(WC_Speed_MS,2) * ((math.sin(math.radians(WC_Theta)) * Draft_Area_Side) + 
                                                                    (math.sin(math.radians((180-WC_Theta) + (Bow_D/2))) * Draft_Area_PB_SB))
            else:
                raise ValueError("Error: Failed to calculate sea current force.\nPlease reenter wind data")
            F_WC_X = F_WC * math.sin(math.radians(WC_Dt))
            F_WC_Y = F_WC * math.cos(math.radians(WC_Dt))

        elif WC_Theta == 180:
                F_WC = 0.5 * Cd * Sea_Density * pow(WC_Speed_MS,2) * math.cos(math.radians(Bow_D/2))*Draft_Area_PB_SB * 2
                F_WC_X = F_WC * math.sin(math.radians(WC_Dt))
                F_WC_Y = F_WC * math.cos(math.radians(WC_Dt))    

        else :
            raise ValueError("Error: Failed to calculate sea current force.\nPlease reenter wind data")

        return F_WC_X, F_WC_Y


    def current_force_calculate(self):
        #Ship Data
        try:
            try:
                self.Ship_Name = str(self.entry_1.get())
            except:
                raise ValueError("Error", "Ship name Error : Please insert alphabet")
            try:    
                self.Ship_D = float(self.entry_2.get())
            except:
                raise ValueError("Error", "Ship direction Error : Please insert number")
            
            #Ship_D_Stern = Ship_D+90

            try:
                self.Ship_Depth = float(self.entry_20.get())
            except:
                raise ValueError("Error", "Ship depth Error : Please insert number")

            try:
                self.Ship_LOA = float(self.entry_3.get())
                # Check if the required fields have valid values
                if self.Ship_LOA <= 0 :
                    # Show an error message if the values are not valid
                    raise ValueError("Error", "Invalid input values. Please enter positive numbers.")
                    return
            except:
                raise ValueError("Error", "Ship LOA Error : Please insert number")

            try:
                self.Ship_Beam = float(self.entry_4.get())
                Half_Ship_Beam = self.Ship_Beam/2
            except:
                raise ValueError("Error", "Ship beam Error : Please insert number")

            try:
                self.LBP = float(self.entry_5.get())
        
            except:
                raise ValueError("Error", "Ship PBL Error : Please insert number")
                
            #Draft
            try:
                self.Draft = float(self.entry_6.get())
            except:
                raise ValueError("Error", "Ship draft Error : Please insert number")

            Lf = self.Ship_LOA-self.LBP

            #Freeboard
            try:
                self.Freeboard = self.Ship_Depth-self.Draft
                self.entry_7.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_7.delete(0, "end")
                self.entry_7.insert(0, "{:.3f}".format(self.Freeboard))
                self.entry_7.configure(state="readonly") # Set the entry back to readonly state 
            except:
                raise ValueError("Error", "Ship Freeboard Error : Please insert Height/Draft number")


            Freeboard_Area_Side = self.Freeboard*self.LBP #Side Area
            Freeboard_Area_Stern = self.Freeboard*self.Ship_Beam
            Freeboard_Area_PB_SB = math.sqrt(pow(Lf,2)+pow(Half_Ship_Beam,2))*self.Freeboard #Ship Bow Area

            PB_SB_D = (abs(math.degrees(math.atan(Lf/Half_Ship_Beam))) + 90)
            Bow_D = (180 - abs(math.degrees(math.atan(Lf/Half_Ship_Beam)))*2)

            #Ship Displacement
            try:
                self.Displacement = float(self.entry_8.get())
            except:
                raise ValueError("Error", "Ship Displacement Error : Please insert number")
        
            #Static Variable    
            Air_Pressure = 1.225
            Sea_Density = 1025
            Cd = 2.1
            g = 9.806

            #compare ship size by displacement
            if self.Displacement < 10000:
                Max_Wind_Speed_MS = 14*0.514444
                Max_Wave_Height = 1.4 #m
                Max_Wave_Speed_MS = 0.85 * Max_Wind_Speed_MS
                Max_WC_Speed_MS = 1*0.514444

            elif self.Displacement < 50000:
                Max_Wind_Speed_MS = 17*0.514444
                Max_Wave_Height = 1.7 #m
                Max_Wave_Speed_MS = 0.85 * Max_Wind_Speed_MS
                Max_WC_Speed_MS = 1*0.514444

            else:
                Max_Wind_Speed_MS = 20*0.514444
                Max_Wave_Height = 2 #m
                Max_Wave_Speed_MS = 0.85 * Max_Wind_Speed_MS
                Max_WC_Speed_MS = 1*0.514444

            #Wind Load
            try:
                self.Wind_Speed_K = float(self.entry_9.get())

                Wind_Speed_MS = self.Wind_Speed_K*0.514444
            except:
                raise ValueError("Error", "Wind speed Error : Please insert number")

            try:
                self.Wind_D = float(self.entry_10.get())

                self.Wave_D = self.Wind_D

                self.entry_13.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_13.delete(0, "end")
                self.entry_13.insert(0, str("{:.3f}".format(self.Wave_D)))
                self.entry_13.configure(state="readonly") # Set the entry back to readonly state

            except:
                raise ValueError("Error", "Wind direction Error : Please insert number")


            F_Wind_X, F_Wind_Y = self.wind_calculate(self.Wind_D,Air_Pressure,Wind_Speed_MS,Freeboard_Area_Stern,Freeboard_Area_Side,
                                                     Freeboard_Area_PB_SB,Bow_D, self.Ship_D)        
            Max_F_Wind_X, Max_F_Wind_Y = self.wind_calculate(self.Wind_D,Air_Pressure,Max_Wind_Speed_MS,Freeboard_Area_Stern,Freeboard_Area_Side,
                                                             Freeboard_Area_PB_SB,Bow_D, self.Ship_D)

            #Wave Force
            try:
                self.Wave_Height = 0.1944 * Wind_Speed_MS #m
                self.entry_12.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_12.delete(0, "end")
                self.entry_12.insert(0, str("{:.3f}".format(self.Wave_Height)))
                self.entry_12.configure(state="readonly") # Set the entry back to readonly state

                Wave_Speed_MS = 0.85 * Wind_Speed_MS

                """T_Period = 0.85 * Wind_Speed_MS
                self.lineEdit_14.setText(str("{:.3f}".format(T_Period)))

                Wave_Length = (g * pow(T_Period,2)) / (2 * math.pi)
                Wave_Speed_MS = Wave_Length/T_Period"""

                self.Wave_Speed_K = Wave_Speed_MS * 1.94384

                self.entry_11.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_11.delete(0, "end")
                self.entry_11.insert(0, str("{:.3f}".format(self.Wave_Speed_K)))
                self.entry_11.configure(state="readonly") # Set the entry back to readonly state

            except:
                raise ValueError("Error", "Wave data Error : Please insert wind data")

            F_Wave_X, F_Wave_Y = self.wave_calculate(self.Wave_D,Sea_Density,Cd,self.Wave_Height,Wave_Speed_MS,Bow_D,self.Ship_D,
                                                     self.Ship_LOA,Lf,self.Ship_Beam,Half_Ship_Beam)
            Max_F_Wave_X, Max_F_Wave_Y = self.wave_calculate(self.Wave_D,Sea_Density,Cd,Max_Wave_Height,Max_Wave_Speed_MS,Bow_D,self.Ship_D,
                                                             self.Ship_LOA,Lf,self.Ship_Beam,Half_Ship_Beam)

            #Underwater Current Force
            
            # Read data from CSV file
            excel_file_path = './Sea Height Pak Nam Rayong 2023.csv'
            watertide_data = pd.read_csv(excel_file_path)

            # Combine 'Date' and 'Time' columns into a new 'DateTime' column
            watertide_data['DateTime'] = pd.to_datetime(watertide_data['Date'] + ' ' + watertide_data['Time'])

            # Drop the original 'Date' and 'Time' columns
            watertide_data = watertide_data.drop(['Date', 'Time'], axis=1)

            # Display the resulting DataFrame
            #print(watertide_data['DateTime'])

            # Get the current time
            self.current_time = datetime.now()
            # Calculate the time one hour ago
            one_hour_ago = self.current_time - timedelta(hours=1)
            two_hour_ago = self.current_time - timedelta(hours=2)
            one_hour_later = self.current_time + timedelta(hours=1)

            # Filter data for the last 1 hour
            current_sea_height_past = watertide_data[(watertide_data['DateTime'] >= one_hour_ago) & (watertide_data['DateTime'] <= self.current_time)]
            current_sea_height_future = watertide_data[(watertide_data['DateTime'] >= self.current_time) & (watertide_data['DateTime'] <= one_hour_later)]

            delcurrent_time = self.current_time - current_sea_height_past['DateTime']

            if (delcurrent_time < timedelta(minutes=30)).any():
                current_sea_height = current_sea_height_past
                one_hour_ago_sea_height = watertide_data[(watertide_data['DateTime'] >= two_hour_ago) & (watertide_data['DateTime'] <= one_hour_ago)]
            elif (delcurrent_time >= timedelta(minutes=30)).any():
                current_sea_height = current_sea_height_future
                one_hour_ago_sea_height = watertide_data[(watertide_data['DateTime'] >= one_hour_ago) & (watertide_data['DateTime'] <= self.current_time)]

            self.change_rate = ((current_sea_height['Sea Height'].iloc[0] - one_hour_ago_sea_height['Sea Height'].iloc[0]))

            self.entry_14.configure(state="normal") # Set the entry to normal state to allow modification
            self.entry_14.delete(0, "end")
            self.entry_14.insert(0, str("{:.1f}".format(self.change_rate)))
            self.entry_14.configure(state="readonly") # Set the entry back to readonly state
            
            try:
                self.WC_Height_Change = float(self.entry_14.get())
                self.WC_Speed_K = self.WC_Height_Change * 5

                self.entry_15.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_15.delete(0, "end")
                self.entry_15.insert(0, str("{:.3f}".format(self.WC_Speed_K)))
                self.entry_15.configure(state="readonly") # Set the entry back to readonly state

                WC_Speed_MS = self.WC_Speed_K*0.514444
                
            except:
                raise ValueError("Error", "Sea current Error : Please insert wind data")

            try:
                if self.WC_Height_Change > 0:
                    self.WC_D = 115
                elif self.WC_Height_Change < 0:
                    self.WC_D = 295
                else:
                    self.WC_D = 0

                self.entry_16.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_16.delete(0, "end")
                self.entry_16.insert(0, str("{:.3f}".format(self.WC_D)))
                self.entry_16.configure(state="readonly") # Set the entry back to readonly state

            except:
                raise ValueError("Error", "Current direction Error : Please insert number")
        
            F_WC_X, F_WC_Y = self.wc_calculate(self.WC_D,self.Wave_Height,Sea_Density,Cd,WC_Speed_MS,Bow_D,self.Ship_D,self.Draft,
                                               self.LBP,Lf,self.Ship_Beam,Half_Ship_Beam)
            Max_F_WC_X, Max_F_WC_Y = self.wc_calculate(self.WC_D,self.Wave_Height,Sea_Density,Cd,Max_WC_Speed_MS,Bow_D,self.Ship_D,self.Draft,
                                                       self.LBP,Lf,self.Ship_Beam,Half_Ship_Beam)

            #Calculate Total Force and Direction
            F_X = F_Wind_X + F_Wave_X + F_WC_X
            F_Y = F_Wind_Y + F_Wave_Y + F_WC_Y
            self.F_Total = math.sqrt(pow(F_X,2)+pow(F_Y,2))/10000
            
            # Change Direction to degree(s) from radians (x/2pi)
            if F_X == 0 and F_Y > 0:
                F_Total_Dt = 0
                self.F_Total_D = 180
            elif F_X > 0 and F_Y > 0:
                F_Total_Dt = math.degrees(math.atan2(F_X, F_Y))
                self.F_Total_D = F_Total_Dt + 180
            elif F_X > 0 and F_Y == 0:
                F_Total_Dt = 90
                self.F_Total_D = 270
            elif F_X > 0 and F_Y < 0:
                F_Total_Dt = math.degrees(math.atan2(F_X, F_Y))
                self.F_Total_D = F_Total_Dt + 180
            elif F_X == 0 and F_Y < 0:
                F_Total_Dt = 180
                self.F_Total_D = 0
            elif F_X < 0 and F_Y < 0:
                F_Total_Dt = math.degrees(math.atan2(F_X, F_Y)) +360
                self.F_Total_D = F_Total_Dt - 180
            elif F_X < 0 and F_Y == 0:
                F_Total_Dt = 270
                self.F_Total_D = 90
            elif F_X < 0 and F_Y > 0:
                F_Total_Dt = math.degrees(math.atan2(F_X, F_Y)) +360
                self.F_Total_D = F_Total_Dt - 180
            else :
                raise ValueError("Error : Failed to calculate total force.\nPlease reinsert data.")

            self.entry_17.configure(state="normal") # Set the entry to normal state to allow modification
            self.entry_17.delete(0, "end")
            self.entry_17.insert(0, str("{:.3f}".format(self.F_Total)))                
            self.entry_17.configure(state="readonly") # Set the entry back to readonly state

            self.result_label_1.configure(text=f"Total Force : {self.F_Total:.2f} Ton")
            self.result_label_4.configure(text=f"Total Force : {self.F_Total:.2f} Ton")

            self.entry_18.configure(state="normal") # Set the entry to normal state to allow modification
            self.entry_18.delete(0, "end")
            self.entry_18.insert(0, str("{:.3f}".format(self.F_Total_D)))             
            self.entry_18.configure(state="readonly") # Set the entry back to readonly state

            self.result_label_2.configure(text=f"Direction : {self.F_Total_D:.2f} Deg.")
            self.result_label_5.configure(text=f"Direction : {self.F_Total_D:.2f} Deg.")                

            #Calculate Maximum Total Force and Direction
            Max_F_X = Max_F_Wind_X + Max_F_Wave_X + Max_F_WC_X
            Max_F_Y = Max_F_Wind_Y + Max_F_Wave_Y + Max_F_WC_Y
            self.Max_F_Total = math.sqrt(pow(Max_F_X,2)+pow(Max_F_Y,2))/10000

            # Change Direction to degree(s) from radians (x/2pi)
            if Max_F_X == 0 and Max_F_Y > 0:
                Max_F_Total_Dt = 0
                Max_F_Total_D = 180
            elif Max_F_X > 0 and Max_F_Y > 0:
                Max_F_Total_Dt = math.degrees(math.atan2(F_X, F_Y))
                Max_F_Total_D = Max_F_Total_Dt + 180
            elif Max_F_X > 0 and Max_F_Y == 0:
                Max_F_Total_Dt = 90
                Max_F_Total_D = 270
            elif Max_F_X > 0 and Max_F_Y < 0:
                Max_F_Total_Dt = math.degrees(math.atan2(F_X, F_Y))
                Max_F_Total_D = Max_F_Total_Dt + 180
            elif Max_F_X == 0 and Max_F_Y < 0:
                Max_F_Total_Dt = 180
                Max_F_Total_D = 0
            elif Max_F_X < 0 and Max_F_Y < 0:
                Max_F_Total_Dt = math.degrees(math.atan2(F_X, F_Y)) +360
                Max_F_Total_D = Max_F_Total_Dt - 180
            elif Max_F_X < 0 and Max_F_Y == 0:
                Max_F_Total_Dt = 270
                Max_F_Total_D = 90
            elif Max_F_X < 0 and Max_F_Y > 0:
                Max_F_Total_Dt = math.degrees(math.atan2(F_X, F_Y)) +360
                Max_F_Total_D = Max_F_Total_Dt - 180
            else :
                raise ValueError("Error : Failed to calculate maximum total force.")

            # Calculate Risk color indication
            # Ship_Weight = float(input("Please insert Ship Weight (Ton(s)) : ")) #Change to LOA*Draft

            risk_color = 0

            if 0 < self.F_Total < (0.04063 * self.Max_F_Total):
                risk_color = 'green'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "Very Safe (0)")           
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state
                mag = 0.2
                # Draw a red bar
                self.update_colored_bar(30, 10, 40, 30, "green")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            elif self.F_Total < (0.15560 * self.Max_F_Total):
                risk_color = 'limegreen'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "Safe (1)")         
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state 
                mag = 0.3
                self.update_colored_bar(30, 10, 80, 30, "limegreen")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            elif self.F_Total < (0.32608 * self.Max_F_Total):
                risk_color = 'greenyellow'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "A Little Safe (2)")         
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state                     
                mag = 0.45
                self.update_colored_bar(30, 10, 120, 30, "greenyellow")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            elif self.F_Total < (0.53611 * self.Max_F_Total):
                risk_color = 'yellow'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "Caution (3)")        
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state     
                mag = 0.6
                self.update_colored_bar(30, 10, 160, 30, "yellow")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            elif self.F_Total < (0.77928 * self.Max_F_Total):
                risk_color = 'darkorange'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "Not Safe (4)")
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state                      
                mag = 0.8
                self.update_colored_bar(30, 10, 200, 30, "darkorange")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            elif self.F_Total <= (self.Max_F_Total) or self.F_Total > (self.Max_F_Total):
                risk_color = 'red'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "Danger (5)")
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state                      
                mag = 1
                self.update_colored_bar(30, 10, 240, 30, "red")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            else:
                raise ValueError("Error : Failed to calculate risk estimation.\nPlease reinsert data")

            # Plot circular data
            self.circular_plot_widget_tab2.plot_circular_data_main(self.Ship_D, self.F_Total_D, self.F_Total, self.Max_F_Total, risk_color, mag)
            self.circular_plot_widget_tab3.plot_circular_data_backup(self.Ship_D, self.Wind_D, self.WC_D, self.F_Total_D, self.F_Total, self.Max_F_Total, risk_color, mag)

        except ValueError as e:
            # Handle other exceptions if necessary
            error_message = f"Failed to calculate overall calculation.\nValue Error : {str(e)}"
            tkinter.messagebox.showerror("Error", error_message)            
        except Exception as e:
            # Handle other exceptions if necessary
            error_message = f"Failed to calculate overall calculation.\nError: {str(e)}"
            tkinter.messagebox.showerror("Error", error_message)

        max_rows = 1000
        csv_file_path = './Output_Real_Time_Database.csv'

        # Read all rows from the CSV file
        with open(csv_file_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            all_rows = list(csv_reader)

        # Increment Item_No based on the existing rows
        Item_No = len(all_rows)

        new_row = [Item_No, self.current_time.strftime("%Y-%m-%d %H:%M:%S"), 'W.3A', self.Ship_Name, str("{:.3f}".format(self.Ship_D)),
                float("{:.3f}".format(self.Ship_LOA)), ("{:.3f}".format(self.Ship_Beam)),
                float("{:.3f}".format(self.LBP)), str("{:.3f}".format(self.Ship_Depth)),
                float("{:.3f}".format(self.Draft)),
                float("{:.3f}".format(self.Freeboard)),
                float("{:.3f}".format(self.Displacement)),
                float("{:.3f}".format(self.Wind_Speed_K)),
                float("{:.3f}".format(self.Wind_D)),
                float("{:.3f}".format(self.Wave_Height)),
                float("{:.3f}".format(self.Wave_Speed_K)),
                float("{:.3f}".format(self.Wave_D)),
                float("{:.3f}".format(self.WC_Speed_K)),
                float("{:.3f}".format(self.WC_D)),
                float("{:.3f}".format(self.F_Total)),
                float("{:.3f}".format(self.F_Total_D)),
                self.entry_19.get()]

        # Open the CSV file in append mode
        with open(csv_file_path, 'a', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)

            # Write the new row
            csv_writer.writerow(new_row)

        """# Check if the number of rows exceeds the maximum
        if len(all_rows) > max_rows:
            # Open the CSV file in write mode
            with open(csv_file_path, 'w', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)

                # Write all rows except the oldest ones
                csv_writer.writerows(all_rows[-max_rows:])"""



    def first_forecast_force_calculate(self):
        #Ship Data
        try:
            try:
                self.Ship_Name = str(self.entry_1.get())
            except:
                raise ValueError("Error", "Ship name Error : Please insert alphabet")
            try:    
                self.Ship_D = float(self.entry_2.get())
            except:
                raise ValueError("Error", "Ship direction Error : Please insert number")
            
            #Ship_D_Stern = Ship_D+90

            try:
                self.Ship_Depth = float(self.entry_20.get())
            except:
                raise ValueError("Error", "Ship depth Error : Please insert number")

            try:
                self.Ship_LOA = float(self.entry_3.get())
                # Check if the required fields have valid values
                if self.Ship_LOA <= 0 :
                    # Show an error message if the values are not valid
                    raise ValueError("Error", "Invalid input values. Please enter positive numbers.")
                    return
            except:
                raise ValueError("Error", "Ship LOA Error : Please insert number")

            try:
                self.Ship_Beam = float(self.entry_4.get())
                Half_Ship_Beam = self.Ship_Beam/2
            except:
                raise ValueError("Error", "Ship beam Error : Please insert number")

            try:
                self.LBP = float(self.entry_5.get())
        
            except:
                raise ValueError("Error", "Ship PBL Error : Please insert number")
                
            #Draft
            try:
                self.Draft = float(self.entry_6.get())
            except:
                raise ValueError("Error", "Ship draft Error : Please insert number")

            Lf = self.Ship_LOA-self.LBP

            #Freeboard
            try:
                self.Freeboard = self.Ship_Depth-self.Draft
                self.entry_7.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_7.delete(0, "end")
                self.entry_7.insert(0, "{:.3f}".format(self.Freeboard))
                self.entry_7.configure(state="readonly") # Set the entry back to readonly state 
            except:
                raise ValueError("Error", "Ship Freeboard Error : Please insert Height/Draft number")


            Freeboard_Area_Side = self.Freeboard*self.LBP #Side Area
            Freeboard_Area_Stern = self.Freeboard*self.Ship_Beam
            Freeboard_Area_PB_SB = math.sqrt(pow(Lf,2)+pow(Half_Ship_Beam,2))*self.Freeboard #Ship Bow Area

            PB_SB_D = (abs(math.degrees(math.atan(Lf/Half_Ship_Beam))) + 90)
            Bow_D = (180 - abs(math.degrees(math.atan(Lf/Half_Ship_Beam)))*2)

            #Ship Displacement
            try:
                self.Displacement = float(self.entry_8.get())
            except:
                raise ValueError("Error", "Ship Displacement Error : Please insert number")
        
            #Static Variable    
            Air_Pressure = 1.225
            Sea_Density = 1025
            Cd = 2.1
            g = 9.806

            #compare ship size by displacement
            if self.Displacement < 10000:
                Max_Wind_Speed_MS = 14*0.514444
                Max_Wave_Height = 1.4 #m
                Max_Wave_Speed_MS = 0.85 * Max_Wind_Speed_MS
                Max_WC_Speed_MS = 1*0.514444

            elif self.Displacement < 50000:
                Max_Wind_Speed_MS = 17*0.514444
                Max_Wave_Height = 1.7 #m
                Max_Wave_Speed_MS = 0.85 * Max_Wind_Speed_MS
                Max_WC_Speed_MS = 1*0.514444

            else:
                Max_Wind_Speed_MS = 20*0.514444
                Max_Wave_Height = 2 #m
                Max_Wave_Speed_MS = 0.85 * Max_Wind_Speed_MS
                Max_WC_Speed_MS = 1*0.514444

            #Wind Load
            # Make an API request to retrieve wind data
            #id = city id and appid = openweathermap api user key id
            #city_id = 1607017
            #appid = 6ad9b22b11cd28e6f397612bea13060d (Example appid={api_key})
            api_url = f'https://api.openweathermap.org/data/2.5/forecast?id=1607017&appid={self.api_key}&units=metric'
            response = requests.get(api_url)

            # Check if the request was successful
            if response.status_code == 200:
                weather_data = response.json()
                cur_datetime = datetime.now()

                # Initialize variables to hold the selected data
                selected_api_data = None

                # Compare the API datetime with the current datetime
                for first_forecast in weather_data["list"]:
                    api_datetime_str = first_forecast["dt_txt"]
                    api_datetime = datetime.strptime(api_datetime_str, "%Y-%m-%d %H:%M:%S")

                    # Set a time difference for comparison (e.g., three hours)
                    time_difference = api_datetime - cur_datetime

                    # Check if the API datetime is approximately three hours in the future
                    if 0 <= time_difference.total_seconds() <= 3 * 3600:
                        selected_api_data = first_forecast
                        break

            else:
                raise ValueError(f"Failed to retrieve wind data. Status code: {response.status_code}")
                        
            # Check if any matching data was found
            if selected_api_data:
                # Access and use the data from the selected_api_data dictionary
                Wind_Speed_MS = selected_api_data["wind"]["speed"]
                self.Wind_D = float(selected_api_data["wind"]["deg"])
                            
                self.Wind_Speed_K = Wind_Speed_MS * 1.94384
                self.entry_9.delete(0, "end")
                self.entry_9.insert(0, "{:.3f}".format(self.Wind_Speed_K))
                self.entry_10.delete(0, "end")
                self.entry_10.insert(0, "{:.3f}".format(self.Wind_D))
                self.Wave_D = self.Wind_D
                self.entry_13.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_13.delete(0, "end")
                self.entry_13.insert(0, "{:.3f}".format(self.Wave_D))
                self.entry_13.configure(state="readonly") # Set the entry back to readonly state 

            else:
                # Show a message if no matching data is found
                raise ValueError("Error", "No matching data found for approximately three hours from the current time.")


            F_Wind_X, F_Wind_Y = self.wind_calculate(self.Wind_D,Air_Pressure,Wind_Speed_MS,Freeboard_Area_Stern,Freeboard_Area_Side,Freeboard_Area_PB_SB,Bow_D,self.Ship_D)
            
            Max_F_Wind_X, Max_F_Wind_Y = self.wind_calculate(self.Wind_D,Air_Pressure,Max_Wind_Speed_MS,Freeboard_Area_Stern,Freeboard_Area_Side,
                                                             Freeboard_Area_PB_SB,Bow_D,self.Ship_D)

            #Wave Force
            try:
                self.Wave_Height = 0.1944 * Wind_Speed_MS #m
                self.entry_12.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_12.delete(0, "end")
                self.entry_12.insert(0, str("{:.3f}".format(self.Wave_Height)))
                self.entry_12.configure(state="readonly") # Set the entry back to readonly state

                Wave_Speed_MS = 0.85 * Wind_Speed_MS

                """T_Period = 0.85 * Wind_Speed_MS
                self.lineEdit_14.setText(str("{:.3f}".format(T_Period)))

                Wave_Length = (g * pow(T_Period,2)) / (2 * math.pi)
                Wave_Speed_MS = Wave_Length/T_Period"""

                self.Wave_Speed_K = Wave_Speed_MS * 1.94384

                self.entry_11.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_11.delete(0, "end")
                self.entry_11.insert(0, str("{:.3f}".format(self.Wave_Speed_K)))
                self.entry_11.configure(state="readonly") # Set the entry back to readonly state

            except:
                raise ValueError("Error", "Wave data Error : Please insert wind data")

            F_Wave_X, F_Wave_Y = self.wave_calculate(self.Wave_D,Sea_Density,Cd,self.Wave_Height,Wave_Speed_MS,Bow_D,self.Ship_D,self.Ship_LOA,Lf,self.Ship_Beam,Half_Ship_Beam)
            Max_F_Wave_X, Max_F_Wave_Y = self.wave_calculate(self.Wave_D,Sea_Density,Cd,Max_Wave_Height,Max_Wave_Speed_MS,Bow_D,self.Ship_D,self.Ship_LOA,Lf,self.Ship_Beam,Half_Ship_Beam)

            #Underwater Current Force

            # Read data from CSV file
            excel_file_path = './Sea Height Pak Nam Rayong 2023.csv'
            watertide_data = pd.read_csv(excel_file_path)

            # Combine 'Date' and 'Time' columns into a new 'DateTime' column
            watertide_data['DateTime'] = pd.to_datetime(watertide_data['Date'] + ' ' + watertide_data['Time'])

            # Drop the original 'Date' and 'Time' columns
            watertide_data = watertide_data.drop(['Date', 'Time'], axis=1)

            # Display the resulting DataFrame
            #print(watertide_data['DateTime'])

            # Get the current time
            self.current_time = datetime.now()

            # Calculate the time one hour ago
            one_hour_ago = self.current_time - timedelta(hours=1)
            two_hour_ago = self.current_time - timedelta(hours=2)
            one_hour_later = self.current_time + timedelta(hours=1)

            # Filter data for the last 1 hour
            current_sea_height_past = watertide_data[(watertide_data['DateTime'] >= one_hour_ago) & (watertide_data['DateTime'] <= self.current_time)]
            current_sea_height_future = watertide_data[(watertide_data['DateTime'] >= self.current_time) & (watertide_data['DateTime'] <= one_hour_later)]

            delcurrent_time = self.current_time - current_sea_height_past['DateTime']

            if (delcurrent_time < timedelta(minutes=30)).any():
                current_sea_height = current_sea_height_past
                one_hour_ago_sea_height = watertide_data[(watertide_data['DateTime'] >= two_hour_ago) & (watertide_data['DateTime'] <= one_hour_ago)]
            elif (delcurrent_time >= timedelta(minutes=30)).any():
                current_sea_height = current_sea_height_future
                one_hour_ago_sea_height = watertide_data[(watertide_data['DateTime'] >= one_hour_ago) & (watertide_data['DateTime'] <= self.current_time)]

            self.change_rate = ((current_sea_height['Sea Height'].iloc[0] - one_hour_ago_sea_height['Sea Height'].iloc[0]))

            self.entry_14.configure(state="normal") # Set the entry to normal state to allow modification
            self.entry_14.delete(0, "end")
            self.entry_14.insert(0, str("{:.1f}".format(self.change_rate)))
            self.entry_14.configure(state="readonly") # Set the entry back to readonly state

            try:
                self.WC_Height_Change = float(self.entry_14.get())
                self.WC_Speed_K = self.WC_Height_Change * 5

                self.entry_15.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_15.delete(0, "end")
                self.entry_15.insert(0, str("{:.3f}".format(self.WC_Speed_K)))
                self.entry_15.configure(state="readonly") # Set the entry back to readonly state

                WC_Speed_MS = self.WC_Speed_K*0.514444
                
            except:
                raise ValueError("Error", "Sea current Error : Please insert wind data")

            try:
                if self.WC_Height_Change > 0:
                    self.WC_D = 115
                elif self.WC_Height_Change < 0:
                    self.WC_D = 295
                else:
                    self.WC_D = 0

                self.entry_16.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_16.delete(0, "end")
                self.entry_16.insert(0, str("{:.3f}".format(self.WC_D)))
                self.entry_16.configure(state="readonly") # Set the entry back to readonly state

            except:
                raise ValueError("Error", "Current direction Error : Please insert number")
        
            F_WC_X, F_WC_Y = self.wc_calculate(self.WC_D,self.Wave_Height,Sea_Density,Cd,WC_Speed_MS,Bow_D,self.Ship_D,self.Draft,
                                               self.LBP,Lf,self.Ship_Beam,Half_Ship_Beam)
            Max_F_WC_X, Max_F_WC_Y = self.wc_calculate(self.WC_D,self.Wave_Height,Sea_Density,Cd,Max_WC_Speed_MS,Bow_D,self.Ship_D,self.Draft,
                                                       self.LBP,Lf,self.Ship_Beam,Half_Ship_Beam)

            #Calculate Total Force and Direction
            F_X = F_Wind_X + F_Wave_X + F_WC_X
            F_Y = F_Wind_Y + F_Wave_Y + F_WC_Y
            self.F_Total = math.sqrt(pow(F_X,2)+pow(F_Y,2))/10000
            
            # Change Direction to degree(s) from radians (x/2pi)
            if F_X == 0 and F_Y > 0:
                F_Total_Dt = 0
                self.F_Total_D = 180
            elif F_X > 0 and F_Y > 0:
                F_Total_Dt = math.degrees(math.atan2(F_X, F_Y))
                self.F_Total_D = F_Total_Dt + 180
            elif F_X > 0 and F_Y == 0:
                F_Total_Dt = 90
                self.F_Total_D = 270
            elif F_X > 0 and F_Y < 0:
                F_Total_Dt = math.degrees(math.atan2(F_X, F_Y))
                self.F_Total_D = F_Total_Dt + 180
            elif F_X == 0 and F_Y < 0:
                F_Total_Dt = 180
                self.F_Total_D = 0
            elif F_X < 0 and F_Y < 0:
                F_Total_Dt = math.degrees(math.atan2(F_X, F_Y)) +360
                self.F_Total_D = F_Total_Dt - 180
            elif F_X < 0 and F_Y == 0:
                F_Total_Dt = 270
                self.F_Total_D = 90
            elif F_X < 0 and F_Y > 0:
                F_Total_Dt = math.degrees(math.atan2(F_X, F_Y)) +360
                self.F_Total_D = F_Total_Dt - 180
            else :
                raise ValueError("Error : Failed to calculate total force.\nPlease reinsert data.")

            self.entry_17.configure(state="normal") # Set the entry to normal state to allow modification
            self.entry_17.delete(0, "end")
            self.entry_17.insert(0, str("{:.3f}".format(self.F_Total)))                
            self.entry_17.configure(state="readonly") # Set the entry back to readonly state

            self.result_label_1.configure(text=f"Total Force : {self.F_Total:.2f} Ton")
            self.result_label_4.configure(text=f"Total Force : {self.F_Total:.2f} Ton")

            self.entry_18.configure(state="normal") # Set the entry to normal state to allow modification
            self.entry_18.delete(0, "end")
            self.entry_18.insert(0, str("{:.3f}".format(self.F_Total_D)))             
            self.entry_18.configure(state="readonly") # Set the entry back to readonly state

            self.result_label_2.configure(text=f"Total Force : {self.F_Total_D:.2f} Deg")
            self.result_label_5.configure(text=f"Total Force : {self.F_Total_D:.2f} Deg")
                           

            #Calculate Maximum Total Force and Direction
            Max_F_X = Max_F_Wind_X + Max_F_Wave_X + Max_F_WC_X
            Max_F_Y = Max_F_Wind_Y + Max_F_Wave_Y + Max_F_WC_Y
            self.Max_F_Total = math.sqrt(pow(Max_F_X,2)+pow(Max_F_Y,2))/10000

            # Change Direction to degree(s) from radians (x/2pi)
            if Max_F_X == 0 and Max_F_Y > 0:
                Max_F_Total_Dt = 0
                Max_F_Total_D = 180
            elif Max_F_X > 0 and Max_F_Y > 0:
                Max_F_Total_Dt = math.degrees(math.atan2(F_X, F_Y))
                Max_F_Total_D = Max_F_Total_Dt + 180
            elif Max_F_X > 0 and Max_F_Y == 0:
                Max_F_Total_Dt = 90
                Max_F_Total_D = 270
            elif Max_F_X > 0 and Max_F_Y < 0:
                Max_F_Total_Dt = math.degrees(math.atan2(F_X, F_Y))
                Max_F_Total_D = Max_F_Total_Dt + 180
            elif Max_F_X == 0 and Max_F_Y < 0:
                Max_F_Total_Dt = 180
                Max_F_Total_D = 0
            elif Max_F_X < 0 and Max_F_Y < 0:
                Max_F_Total_Dt = math.degrees(math.atan2(F_X, F_Y)) +360
                Max_F_Total_D = Max_F_Total_Dt - 180
            elif Max_F_X < 0 and Max_F_Y == 0:
                Max_F_Total_Dt = 270
                Max_F_Total_D = 90
            elif Max_F_X < 0 and Max_F_Y > 0:
                Max_F_Total_Dt = math.degrees(math.atan2(F_X, F_Y)) +360
                Max_F_Total_D = Max_F_Total_Dt - 180
            else :
                raise ValueError("Error : Failed to calculate maximum total force.")

            # Calculate Risk color indication
            # Ship_Weight = float(input("Please insert Ship Weight (Ton(s)) : ")) #Change to LOA*Draft

            risk_color = 0
            
            if 0 < self.F_Total < (0.04063 * self.Max_F_Total):
                risk_color = 'green'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "Very Safe (0)")           
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state
                mag = 0.2
                # Draw a red bar
                self.update_colored_bar(30, 10, 40, 30, "green")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            elif self.F_Total < (0.15560 * self.Max_F_Total):
                risk_color = 'limegreen'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "Safe (1)")         
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state 
                mag = 0.3
                self.update_colored_bar(30, 10, 80, 30, "limegreen")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            elif self.F_Total < (0.32608 * self.Max_F_Total):
                risk_color = 'greenyellow'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "A Little Safe (2)")         
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state                     
                mag = 0.45
                self.update_colored_bar(30, 10, 120, 30, "greenyellow")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            elif self.F_Total < (0.53611 * self.Max_F_Total):
                risk_color = 'yellow'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "Caution (3)")        
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state     
                mag = 0.6
                self.update_colored_bar(30, 10, 160, 30, "yellow")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            elif self.F_Total < (0.77928 * self.Max_F_Total):
                risk_color = 'darkorange'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "Not Safe (4)")
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state                      
                mag = 0.8
                self.update_colored_bar(30, 10, 200, 30, "darkorange")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            elif self.F_Total <= (self.Max_F_Total) or self.F_Total > (self.Max_F_Total):
                risk_color = 'red'
                self.entry_19.configure(state="normal") # Set the entry to normal state to allow modification
                self.entry_19.delete(0, "end")
                self.entry_19.insert(0, "Danger (5)")
                self.entry_19.configure(state="readonly") # Set the entry back to readonly state                      
                mag = 1
                self.update_colored_bar(30, 10, 240, 30, "red")
                self.result_label_3.configure(text=f"Risk Estimation : {self.entry_19.get()}")
                self.result_label_6.configure(text=f"Risk Estimation : {self.entry_19.get()}")

            else:
                raise ValueError("Error : Failed to calculate risk estimation.\nPlease reinsert data")

            # Plot circular data
            self.circular_plot_widget_tab2.plot_circular_data_main(self.Ship_D, self.F_Total_D, self.F_Total, self.Max_F_Total, risk_color, mag)
            self.circular_plot_widget_tab3.plot_circular_data_backup(self.Ship_D, self.Wind_D, self.WC_D, self.F_Total_D, self.F_Total, self.Max_F_Total, risk_color, mag)

        except ValueError as e:
            # Handle other exceptions if necessary
            error_message = f"Failed to calculate overall calculation.\nValue Error : {str(e)}"
            tkinter.messagebox.showerror("Error", error_message)            
        except Exception as e:
            # Handle other exceptions if necessary
            error_message = f"Failed to calculate overall calculation.\nError: {str(e)}"
            tkinter.messagebox.showerror("Error", error_message)
        
        # list of column names
        max_rows = 1000
        csv_file_path = './Output_API_Database.csv'

        # Read all rows from the CSV file
        with open(csv_file_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            all_rows = list(csv_reader)

        # Clear existing rows in the Treeview
        for item in self.treeview.get_children():
            self.treeview.delete(item)
            
        # Increment Item_No based on the existing rows
        Item_No = len(all_rows)

        new_row = [Item_No, self.current_time.strftime("%Y-%m-%d %H:%M:%S"), 'W.3A', self.Ship_Name, str("{:.3f}".format(self.Ship_D)),
                float("{:.3f}".format(self.Ship_LOA)), ("{:.3f}".format(self.Ship_Beam)),
                float("{:.3f}".format(self.LBP)), str("{:.3f}".format(self.Ship_Depth)),
                float("{:.3f}".format(self.Draft)),
                float("{:.3f}".format(self.Freeboard)),
                float("{:.3f}".format(self.Displacement)),
                float("{:.3f}".format(self.Wind_Speed_K)),
                float("{:.3f}".format(self.Wind_D)),
                float("{:.3f}".format(self.Wave_Height)),
                float("{:.3f}".format(self.Wave_Speed_K)),
                float("{:.3f}".format(self.Wave_D)),
                float("{:.3f}".format(self.WC_Speed_K)),
                float("{:.3f}".format(self.WC_D)),
                float("{:.3f}".format(self.F_Total)),
                float("{:.3f}".format(self.F_Total_D)),
                self.entry_19.get()]

        # Insert new data into the Treeview
        for index, row in enumerate(all_rows):
            self.treeview.insert("", "end", values=row)

        # Check if the number of rows exceeds the maximum
        """if len(all_rows) > max_rows:
            # Open the CSV file in write mode
            with open(csv_file_path, 'w', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)

                # Write all rows except the oldest ones
                csv_writer.writerows(all_rows[-max_rows:])"""
        

    # Add a method to export the graph and data
    def export_graph_data(self):
        row_list = [["Variable", "Value", "Unit"],
                    ["Ship Name", self.Ship_Name],
                    ["Ship/Port Direction", str("{:.3f}".format(self.Ship_D)), " Degree(s)"],
                    ["Ship LOA", str("{:.3f}".format(self.Ship_LOA)), "meter(s)"],
                    ["Ship Beam", str("{:.3f}".format(self.Ship_Beam)), "meter(s)"],
                    ["Ship PBL", str("{:.3f}".format(self.LBP)), "meter(s)"],
                    ["Ship Draft", str("{:.3f}".format(self.Draft)), "meter(s)"],
                    ["Ship Freeboard", str("{:.3f}".format(self.Freeboard)), "meter(s)"],
                    ["Ship Ship Displacement", str("{:.3f}".format(self.Displacement)), "Ton(s)"],
                    ["Wind Speed", str("{:.3f}".format(self.Wind_Speed_K)), "Knot"],
                    ["Wind Direction", str("{:.3f}".format(self.Wind_D)), "Degree(s)"],
                    ["Wave Speed", str("{:.3f}".format(self.Wave_Speed_K)), "Knot"],
                    ["Wave Height", str("{:.3f}".format(self.Wave_Height)), "Meter(s)"],
                    ["Wave Direction",  str("{:.3f}".format(self.Wave_D)), "Degree(s)"],
                    ["Sea Height Change rate", str("{:.3f}".format(self.WC_Height_Change)), "Meter(s)/Hour"],
                    ["Sea Current Speed", str("{:.3f}".format(self.WC_Speed_K)), "Knot"],
                    ["Sea Current Direction", str("{:.3f}".format(self.WC_D)), "Degree(s)"],
                    ["Total Force", str("{:.3f}".format(self.F_Total)), "Ton(s)"],
                    ["Total Force Direction", str("{:.3f}".format(self.F_Total_D)), "Degree(s)"],
                    ["Risk Extimation", self.entry_19.get()]]
        
        # Get the filename from a file dialog
        file_path_3 = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG Images", "*.png"), ("CSV Files", "*.csv")])

        if file_path_3:
            try:
                # Export the graph as an image (e.g., PNG)
                self.circular_plot_widget_tab2.figure.savefig(file_path_3 + ".png", dpi=300)
                self.circular_plot_widget_tab3.figure.savefig(f"{file_path_3}_backup" + ".png", dpi=300)
                # Export the data as a CSV file (modify this part to include your data)
                with open(file_path_3 + ".csv", "w", newline="") as csv_file:
                    csv_writer = csv.writer(csv_file)
                    csv_writer.writerows(row_list)

                # Show a success message or handle errors as needed
                tkinter.messagebox.showinfo("Success", "Export successful!")
                
            except Exception as e:
                #self.result_text.delete(1.0, tkinter.END)
                #self.result_text.insert(tkinter.END, f"An error occurred: {str(e)}")
                error_message = f"Failed to export data.\nError: {str(e)}"
                tkinter.messagebox.showerror("Error", error_message)

if __name__ == "__main__":
    app = App()
    app.mainloop()